In [37]:
import pandas as pd
import re
from sqlalchemy import create_engine

# Extracting Data

## Individual cyber attacks

In [38]:
# Read the CSV file
df_individuals = pd.read_csv('../Original Datasets/individuals_cyber_attacks_europe.csv')
df_ind = df_individuals.copy()
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

In [39]:
# Database connection details
db_user = 'root'  # Replace with your MySQL username
db_password = 'root'  # Replace with your MySQL password
db_host = 'localhost'  # Or your database server hostname
db_name = 'staging_cyber_attacks'  # Replace with your database name
table_name = 'cyber_attacks_individuals'  # Replace with your desired table name

# Create the database engine
engine = create_engine(f'mysql+pymysql://{db_user}:{db_password}@{db_host}/{db_name}')

# Write the DataFrame to the MySQL table
df_individuals.to_sql(table_name, con=engine, if_exists='replace', index=False)

print(f"Data imported successfully to table '{table_name}' in database '{db_name}'")

Data imported successfully to table 'cyber_attacks_individuals' in database 'staging_cyber_attacks'


## Companies cyber attacks

In [40]:
# Read the CSV file
df_companies = pd.read_csv('../Original Datasets/companies_cyber_attacks_europe.csv')
df_comp = df_companies.copy()
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

In [41]:
# Database connection details
db_user = 'root'  # Replace with your MySQL username
db_password = 'root'  # Replace with your MySQL password
db_host = 'localhost'  # Or your database server hostname
db_name = 'staging_cyber_attacks'  # Replace with your database name
table_name = 'cyber_attacks_companies'  # Replace with your desired table name

# Create the database engine
engine = create_engine(f'mysql+pymysql://{db_user}:{db_password}@{db_host}/{db_name}')

# Write the DataFrame to the MySQL table
df_companies.to_sql(table_name, con=engine, if_exists='replace', index=False)

print(f"Data imported successfully to table '{table_name}' in database '{db_name}'")

Data imported successfully to table 'cyber_attacks_companies' in database 'staging_cyber_attacks'


## Active Users

In [42]:
# Read the CSV file
df_users = pd.read_csv('../Original Datasets/active_online_users_europe_2018_2023.csv')
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

In [43]:
# Database connection details
db_user = 'root'  # Replace with your MySQL username
db_password = 'root'  # Replace with your MySQL password
db_host = 'localhost'  # Or your database server hostname
db_name = 'staging_cyber_attacks'  # Replace with your database name
table_name = 'active_online_users'  # Replace with your desired table name

# Create the database engine
engine = create_engine(f'mysql+pymysql://{db_user}:{db_password}@{db_host}/{db_name}')

# Write the DataFrame to the MySQL table
df_users.to_sql(table_name, con=engine, if_exists='replace', index=False)

print(f"Data imported successfully to table '{table_name}' in database '{db_name}'")

Data imported successfully to table 'active_online_users' in database 'staging_cyber_attacks'


# Transforming Data

## Cyber Attacks on Individuals

In [44]:
# Drop rows where all elements are missing
df_ind.dropna(how='all', inplace=True)

In [45]:
df_ind['Timestamp'] = pd.to_datetime(df_ind['Timestamp'], format='%m/%d/%Y %H:%M')

In [46]:
# Fill nulls with 'Not Available'
columns_to_fill = ['Malware Indicators', 'Alerts/Warnings', 'Proxy Information', 'Firewall Logs', 'IDS/IPS Alerts']
df_ind[columns_to_fill] = df_ind[columns_to_fill].fillna('Not Available')

In [47]:
# Drop duplicates
df_ind.drop_duplicates(inplace=True)
df_ind.reset_index(drop=True, inplace=True)

In [48]:
# Validate IP addresses
def is_valid_ip(ip):
    pattern = r"^((25[0-5]|2[0-4][0-9]|[01]?[0-9][0-9]?)\.){3}(25[0-5]|2[0-4][0-9]|[01]?[0-9][0-9]?)$"
    return bool(re.match(pattern, ip))

df_ind['Valid Source IP'] = df_ind['Source IP Address'].apply(is_valid_ip)
df_ind['Valid Destination IP'] = df_ind['Destination IP Address'].apply(is_valid_ip)

In [49]:
# Check for invalid port numbers
invalid_source_ports = df_ind[(df_ind['Source Port'] < 0) | (df_ind['Source Port'] > 65535)]
invalid_destination_ports = df_ind[(df_ind['Destination Port'] < 0) | (df_ind['Destination Port'] > 65535)]

if invalid_source_ports.shape[0] > 0 or invalid_destination_ports.shape[0] > 0:
    print('Invalid port numbers found')
else:
    print('All port numbers are valid')

Invalid port numbers found


In [50]:
# Count invalid IPs
invalid_source_ip_count = df_ind[df_ind['Valid Source IP'] == False].shape[0]
invalid_destination_ip_count = df_ind[df_ind['Valid Destination IP'] == False].shape[0]

print(f"\nNumber of rows with invalid Source IP Address: {invalid_source_ip_count}")
print(f"Number of rows with invalid Destination IP Address: {invalid_destination_ip_count}")


Number of rows with invalid Source IP Address: 0
Number of rows with invalid Destination IP Address: 0


In [51]:
# Extract features from 'Timestamp'
df_ind['Day'] = df_ind['Timestamp'].dt.day
df_ind['Hour of Day'] = df_ind['Timestamp'].dt.hour
df_ind['Month'] = df_ind['Timestamp'].dt.month
df_ind['Year'] = df_ind['Timestamp'].dt.year

In [52]:
def extract_browser(text):
    # Define patterns for common browsers
    patterns = {
        'Chrome': r'Chrome\/[\d.]+',
        'Firefox': r'Firefox\/[\d.]+',
        'Safari': r'Safari\/[\d.]+',
        'Edge': r'Edge\/[\d.]+',
        'MSIE': r'MSIE [\d.]+',
        'Trident': r'Trident\/[\d.]+'  # For older IE versions
    }

    for browser, pattern in patterns.items():
        match = re.search(pattern, text)
        if match:
            return browser
    return 'Unknown'

In [53]:
def extract_os(text):
    # Define patterns for common operating systems
    patterns = {
        'Windows': r'Windows NT [\d.]+',
        'Mac OS X': r'Mac OS X [\d._]+',
        'Linux': r'Linux',
        'Android': r'Android [\d.]+',
        'iOS': r'iPhone|iPad|iPod'
    }

    for os, pattern in patterns.items():
        match = re.search(pattern, text)
        if match:
            return os
    return 'Unknown'

In [54]:
def extract_device_type(text):
    # Define patterns for common device types
    if 'Mobile' in text or 'Android' in text or 'iPhone' in text or 'iPad' in text or 'iPod' in text:
        return 'Mobile'
    elif 'Tablet' in text:
        return 'Tablet'
    else:
        return 'Desktop'

In [55]:
# Apply the functions to the 'Device Information' column
df_ind['Browser'] = df_ind['Device Information'].astype(str).apply(extract_browser)
df_ind['Operating System'] = df_ind['Device Information'].astype(str).apply(extract_os)
df_ind['Device Type'] = df_ind['Device Information'].astype(str).apply(extract_device_type)

In [56]:
df_ind = df_ind.drop('Device Information', axis=1)

In [57]:
df_ind.head()

,Timestamp,Source IP Address,Destination IP Address,Source Port,Destination Port,Protocol,Packet Length,Packet Type,Traffic Type,Payload Data,Malware Indicators,Anomaly Scores,Alerts/Warnings,Attack Type,Attack Signature,Action Taken,Severity Level,User Information,Network Segment,Proxy Information,Firewall Logs,IDS/IPS Alerts,Log Source,Country,Source_Country,Valid Source IP,Valid Destination IP,Day,Hour of Day,Month,Year,Browser,Operating System,Device Type
0,2023-05-30 06:33:00,16.208.61.129,84.42.185.26,17604,17616,ICMP,503,Data,HTTP,Qui natus odio asperiores nam. Optio nobis ius...,IoC Detected,28.67,Not Available,Malware,Known Pattern B,Logged,Low,Reyansh Dugal,Segment A,150.9.97.135,Log Data,Not Available,Server,France,United States,True,True,30,6,5,2023,MSIE,Windows,Desktop
1,2020-08-26 07:08:00,2.13.205.122,66.10.166.120,26446,48166,ICMP,1174,Data,HTTP,Aperiam quos modi officiis veritatis rem. Omni...,IoC Detected,51.50,Not Available,Malware,Known Pattern A,Blocked,Low,Sumer Rana,Segment B,Not Available,Log Data,Not Available,Firewall,Germany,China,True,True,26,7,8,2020,MSIE,Windows,Desktop
2,2022-11-13 08:23:00,182.214.2.177,198.117.46.175,51628,53600,UDP,306,Control,HTTP,Perferendis sapiente vitae soluta. Hic delectu...,IoC Detected,87.42,Alert Triggered,DDoS,Known Pattern B,Ignored,Low,Himmat Karpe,Segment C,114.133.48.179,Log Data,Alert Data,Firewall,Belgium,Iran,True,True,13,8,11,2022,MSIE,Windows,Desktop
3,2023-07-02 10:38:00,211.75.229.120,66.122.35.118,95465,32534,UDP,385,Data,HTTP,Totam maxime beatae expedita explicabo porro l...,Not Available,15.79,Alert Triggered,Malware,Known Pattern B,Blocked,Medium,Fateh Kibe,Segment B,Not Available,Not Available,Alert Data,Firewall,Germany,Turkey,True,True,2,10,7,2023,Firefox,Mac OS X,Desktop
4,2023-07-16 13:11:00,85.94.195.186,66.81.238.90,33491,26646,TCP,1462,Data,DNS,Odit nesciunt dolorem nisi iste iusto. Animi v...,Not Available,0.52,Alert Triggered,DDoS,Known Pattern B,Blocked,Low,Dhanush Chad,Segment C,149.6.110.119,Not Available,Alert Data,Firewall,Germany,Russia,True,True,16,13,7,2023,MSIE,Windows,Desktop


## Cyber Attacks on Companies

In [58]:
# 1. Checking for missing values
missing_values = df_comp.isnull().sum()
missing_values

Incident ID                        0
Company Name                       0
Industry                           0
Attack Type                        0
Date of Incident                   0
Breach Size (No. of Records)    6005
Financial Loss (USD)               0
Country                            0
Attack Vector                      0
Mitigation Time (Days)             0
dtype: int64

In [59]:
df_comp.head()

,Incident ID,Company Name,Industry,Attack Type,Date of Incident,Breach Size (No. of Records),Financial Loss (USD),Country,Attack Vector,Mitigation Time (Days)
0,1,White Inc,Retail,Ransomware,2018-08-27,6525846.0,506815.46,Italy,DDoS Attack,25
1,2,Vasquez PLC,Telecommunications,Ransomware,2018-05-24,9114896.0,1668001.94,Netherlands,Compromised Vendor,29
2,3,Gardner LLC,Retail,Supply Chain Attack,2018-07-04,3412886.0,4073566.14,Belgium,Compromised Vendor,22
3,4,Pena-Kirk,Manufacturing,Ransomware,2018-02-05,5829814.0,3211005.93,Belgium,Compromised Vendor,60
4,5,"Garcia, Butler and Kennedy",Healthcare,Ransomware,2019-04-05,9996201.0,4033042.07,UK,Compromised Vendor,59


In [60]:
# 2. Ensuring correct data types (e.g., 'Date of Incident' to datetime, numeric columns for breach size and financial loss)
df_comp['Date of Incident'] = pd.to_datetime(df_comp['Date of Incident'])
df_comp['Breach Size (No. of Records)'] = pd.to_numeric(df_comp['Breach Size (No. of Records)'])
df_comp['Financial Loss (USD)'] = pd.to_numeric(df_comp['Financial Loss (USD)'])
df_comp['Mitigation Time (Days)'] = pd.to_numeric(df_comp['Mitigation Time (Days)'])


In [61]:
# 3. Standardizing text data (Company Name, Country, and Industry columns)
df_comp['Company Name'] = df_comp['Company Name'].str.title().str.strip()
df_comp['Industry'] = df_comp['Industry'].str.title().str.strip()
df_comp['Country'] = df_comp['Country'].str.title().str.strip()


In [62]:
# 4. Removing duplicates
df_comp = df_comp.drop_duplicates()

In [63]:
# 5. Handling missing values for 'Breach Size (No. of Records)'
median_breach_size = df_comp['Breach Size (No. of Records)'].median()
df_comp['Breach Size (No. of Records)'].fillna(median_breach_size, inplace=True)

C:\Users\JasonGreich\AppData\Local\Temp\ipykernel_9580\381758125.py:3: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_comp['Breach Size (No. of Records)'].fillna(median_breach_size, inplace=True)


In [64]:
# 6. Creating new columns for year, month, and day from 'Date of Incident'
df_comp['Year'] = df_comp['Date of Incident'].dt.year
df_comp['Month'] = df_comp['Date of Incident'].dt.month
df_comp['Day'] = df_comp['Date of Incident'].dt.day


In [65]:
# Display the cleaned data
df_comp.head()

,Incident ID,Company Name,Industry,Attack Type,Date of Incident,Breach Size (No. of Records),Financial Loss (USD),Country,Attack Vector,Mitigation Time (Days),Year,Month,Day
0,1,White Inc,Retail,Ransomware,2018-08-27,6525846.0,506815.46,Italy,DDoS Attack,25,2018,8,27
1,2,Vasquez Plc,Telecommunications,Ransomware,2018-05-24,9114896.0,1668001.94,Netherlands,Compromised Vendor,29,2018,5,24
2,3,Gardner Llc,Retail,Supply Chain Attack,2018-07-04,3412886.0,4073566.14,Belgium,Compromised Vendor,22,2018,7,4
3,4,Pena-Kirk,Manufacturing,Ransomware,2018-02-05,5829814.0,3211005.93,Belgium,Compromised Vendor,60,2018,2,5
4,5,"Garcia, Butler And Kennedy",Healthcare,Ransomware,2019-04-05,9996201.0,4033042.07,Uk,Compromised Vendor,59,2019,4,5


## Active Users

In [66]:
# Convert the 'Date' column to datetime format
df_users['Date'] = pd.to_datetime(df_users['Date'])

In [67]:
# Create new columns for year, month, and day
df_users['Year'] = df_users['Date'].dt.year
df_users['Month'] = df_users['Date'].dt.month
df_users['Day'] = df_users['Date'].dt.day

In [68]:
df_users.head()

,Date,Active Online Users,Year,Month,Day
0,2018-01-01,412000000,2018,1,1
1,2018-01-02,413653325,2018,1,2
2,2018-01-03,412789947,2018,1,3
3,2018-01-04,413882307,2018,1,4
4,2018-01-05,412000000,2018,1,5


## Changing Columns Names

In [69]:
# Rename for individuals dataset
individuals_rename_mapping = {
    'Unnamed: 0': 'ID',
    'Timestamp': 'Timestamp',
    'Source IP Address': 'Source_IP_Address',
    'Destination IP Address': 'Destination_IP_Address',
    'Source Port': 'Source_Port',
    'Destination Port': 'Destination_Port',
    'Protocol': 'Protocol',
    'Packet Length': 'Packet_Length',
    'Packet Type': 'Packet_Type',
    'Traffic Type': 'Traffic_Type',
    'Payload Data': 'Payload_Data',
    'Malware Indicators': 'Malware_Indicators',
    'Anomaly Scores': 'Anomaly_Scores',
    'Alerts/Warnings': 'Alerts_Warnings',
    'Attack Type': 'Attack_Type',
    'Attack Signature': 'Attack_Signature',
    'Action Taken': 'Action_Taken',
    'Severity Level': 'Severity_Level',
    'User Information': 'User_Information',
    'Network Segment': 'Network_Segment',
    'Proxy Information': 'Proxy_Information',
    'Firewall Logs': 'Firewall_Logs',
    'IDS/IPS Alerts': 'IDS_IPS_Alerts',
    'Log Source': 'Log_Source',
    'Country': 'Country',
    'Valid Source IP': 'Valid_Source_IP',
    'Valid Destination IP': 'Valid_Destination_IP',
    'Day': 'Day',
    'Hour of Day': 'Hour_of_Day',
    'Month': 'Month',
    'Year': 'Year',
    'Browser': 'Browser',
    'Operating System': 'Operating_System',
    'Device Type': 'Device_Type'
}

# Rename for companies dataset
companies_rename_mapping = {
    'Unnamed: 0': 'ID',
    'Incident ID': 'Incident_ID',
    'Company Name': 'Company_Name',
    'Industry': 'Industry',
    'Attack Type': 'Attack_Type',
    'Date of Incident': 'Date_of_Incident',
    'Breach Size (No. of Records)': 'Breach_Size',
    'Financial Loss (USD)': 'Financial_Loss',
    'Country': 'Country',
    'Attack Vector': 'Attack_Vector',
    'Mitigation Time (Days)': 'Mitigation_Time',
    'Year': 'Year',
    'Month': 'Month',
    'Day': 'Day'
}

# Rename for active users dataset
active_users_rename_mapping = {
    'Unnamed: 0': 'ID',
    'Date': 'Date',
    'Active Online Users': 'Active_Online_Users',
    'Year': 'Year',
    'Month': 'Month',
    'Day': 'Day'
}

# Apply renaming
df_ind.rename(columns=individuals_rename_mapping, inplace=True)
df_comp.rename(columns=companies_rename_mapping, inplace=True)
df_users.rename(columns=active_users_rename_mapping, inplace=True)

# Save the updated files
df_ind.to_csv('./Transformed_Data/individuals_cyber_attacks_europe.csv', index=False)
df_comp.to_csv('./Transformed_Data/companies_cyber_attacks_europe.csv', index=False)
df_users.to_csv('./Transformed_Data/active_online_users_europe_2018_2023.csv', index=False)

# Loading 

## Loading to transfomation DB

In [70]:
# Database connection details
db_user = 'root'  # Replace with your MySQL username
db_password = 'root'  # Replace with your MySQL password
db_host = 'localhost'  # Or your database server hostname
db_name = 'transformed_cyber_staging'  # Replace with your database name
table_name = 'companies_cyber_attacks'  # Replace with your desired table name

# Create the database engine
engine = create_engine(f'mysql+pymysql://{db_user}:{db_password}@{db_host}/{db_name}')

# Write the DataFrame to the MySQL table
df_comp.to_sql(table_name, con=engine, if_exists='replace', index=False)

print(f"Data imported successfully to table '{table_name}' in database '{db_name}'")

Data imported successfully to table 'companies_cyber_attacks' in database 'transformed_cyber_staging'


In [71]:
# Database connection details
db_user = 'root'  # Replace with your MySQL username
db_password = 'root'  # Replace with your MySQL password
db_host = 'localhost'  # Or your database server hostname
db_name = 'transformed_cyber_staging'  # Replace with your database name
table_name = 'individual_cyber_attacks'  # Replace with your desired table name

# Create the database engine
engine = create_engine(f'mysql+pymysql://{db_user}:{db_password}@{db_host}/{db_name}')

# Write the DataFrame to the MySQL table
df_ind.to_sql(table_name, con=engine, if_exists='replace', index=False)

print(f"Data imported successfully to table '{table_name}' in database '{db_name}'")

Data imported successfully to table 'individual_cyber_attacks' in database 'transformed_cyber_staging'


In [72]:
# Database connection details
db_user = 'root'  # Replace with your MySQL username
db_password = 'root'  # Replace with your MySQL password
db_host = 'localhost'  # Or your database server hostname
db_name = 'transformed_cyber_staging'  # Replace with your database name
table_name = 'active_online_users'  # Replace with your desired table name

# Create the database engine
engine = create_engine(f'mysql+pymysql://{db_user}:{db_password}@{db_host}/{db_name}')

# Write the DataFrame to the MySQL table
df_users.to_sql(table_name, con=engine, if_exists='replace', index=False)

print(f"Data imported successfully to table '{table_name}' in database '{db_name}'")

Data imported successfully to table 'active_online_users' in database 'transformed_cyber_staging'
